In [1]:
import numpy as np
import pandas as pd
import evaluate
import torch
import json
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from torch.nn import CrossEntropyLoss

from preprocessing import climateBUG_reduce_rows



In [2]:
# This should always output true now, but worth checking
print(f"is GPU available: {torch.cuda.is_available()}")

is GPU available: True


In [3]:
# The climate-fever dataset is in a format which the transformers Trainer does not understand
# It must be preprocessed using the functions in preprocessing.py

df = pd.read_json("/home/lukeg/Documents/VS_code/fine_tuning/lxg406/climate_relatedness_classification/data/climateBUG/climateBUG-training-dataset.json")
preprocessed_training_df = climateBUG_reduce_rows(df, rows=10000)

df = pd.read_json("/home/lukeg/Documents/VS_code/fine_tuning/lxg406/climate_relatedness_classification/data/climateBUG/climateBUG-testing-dataset.json")
preprocessed_testing_df = climateBUG_reduce_rows(df, rows=1000)
preprocessed_testing_df.loc[998]


                                              statement  label
0     2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REP...      0
1     Retail     banking     achieved     a     stro...      0
2     Crédit   Agricole   Assurances   captured  fur...      0
3     Our  business  is  also  exposed  to  the  ups...      0
4     Crédit  Agricole  assumed  its   responsibilit...      0
...                                                 ...    ...
9995  Other general administrative expenses (Note 33...      0
9996  Other financial instruments not measured at fa...      0
9997             Revaluation gains/(losses) 191 (1,054)      0
9998  Final balance at 31.12.2014 5,714,956 12,032,8...      0
9999                Tangible assets (218,340) (249,701)      0

[10000 rows x 2 columns]
                                              statement  label
0     2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REP...      0
1     Retail     banking     achieved     a     stro...      0
2     Crédit   Agricole   Ass

statement    

):A@\t6
/,"+,2+
$)
...
label                                                        1
Name: 998, dtype: object

In [4]:
training_dataset = Dataset.from_pandas(preprocessed_training_df)
testing_dataset = Dataset.from_pandas(preprocessed_testing_df)
training_dataset[0]

{'statement': '2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REPORT2 CRÉDIT AGRICOLE S.A.      2015 REGISTRATION DOCUMENT MESSAGE FROM THE CHAIRMAN AND THE CHIEF EXECUTIVE OFFICER MESSAGE FROM THE CHAIRMANAND THE CHIEF EXECUTIVE OFFICERPHILIPPE BRASSAC, CHIEF EXECUTIVE OFFICER AND DOMINIQUE LEFEBVRE, CHAIRMAN OF CRÉDIT AGRICOLE S.A.A robust Group delivering sustainable performanceLeading  a  bank  with  a  EUR  698  billion   loan   book(1)',
 'label': 0}

In [5]:
seed=11
training_dataset = training_dataset.shuffle(seed=seed)
testing_dataset = testing_dataset.shuffle(seed=seed)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")

/home/lukeg/ClimatEnv/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
def custom_tokenize(examples):
    tokenized_output = tokenizer(
        text=examples["statement"],
        max_length=512, 
        padding="max_length", 
        truncation=True)

    return tokenized_output

tokenized_training_dataset = training_dataset.map(custom_tokenize, batched=True)
tokenized_testing_dataset = testing_dataset.map(custom_tokenize, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
tokenized_training_dataset

Dataset({
    features: ['statement', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [9]:

print(tokenized_training_dataset[0]["statement"])
print(tokenized_training_dataset[0]["label"])
print(tokenized_training_dataset[0]["input_ids"])
print(tokenized_training_dataset[0]["attention_mask"])

print(tokenized_testing_dataset[9]["statement"])
print(tokenized_testing_dataset[9]["label"])
print(tokenized_testing_dataset[9]["input_ids"])
print(tokenized_testing_dataset[9]["attention_mask"])

Refer to https://sustainabledevelopment.un.org/ for more information on the Sustainable Development Goals.
1
[0, 48310, 7, 1205, 640, 29, 26661, 19875, 242, 27216, 1757, 4, 879, 4, 1957, 73, 13, 55, 335, 15, 5, 20586, 2717, 20638, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilroberta-base",
    num_labels=2,
    # ignore_mismatched_sizes=True
).to("cuda")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
print(next(model.parameters()).device)

cuda:0


In [12]:
model.gradient_checkpointing_enable()

In [13]:
def calculate_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")

    return {
        "accuracy": accuracy,
        "f1_score": f1,
    }

In [14]:
# This is where we set the hyperparameters
training_args = TrainingArguments(
    output_dir="./results/distilroberta",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=200,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="no",
    save_steps=500,
    fp16=True,                          # Use 16-bit floating point instead of 32 - makes computation faster
    warmup_ratio=0.05,                    # Allows the model to adapt a little
    # gradient_accumulation_steps=2       # Might help with OOM errors, if we have them
    learning_rate=3e-5,
    push_to_hub=False,
)

In [15]:
trainer  = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_training_dataset,
    tokenizer=tokenizer,
    eval_dataset=tokenized_testing_dataset,
    compute_metrics=calculate_metrics,
)

In [16]:
trainer.train()

  0%|          | 0/3750 [00:00<?, ?it/s]

{'loss': 0.693, 'grad_norm': 1.6136739253997803, 'learning_rate': 7.978723404255319e-06, 'epoch': 0.04}
{'loss': 0.5066, 'grad_norm': inf, 'learning_rate': 1.579787234042553e-05, 'epoch': 0.08}
{'loss': 0.4339, 'grad_norm': 20.080265045166016, 'learning_rate': 2.345744680851064e-05, 'epoch': 0.12}
{'loss': 0.3832, 'grad_norm': 3.3227107524871826, 'learning_rate': 2.9924199887703537e-05, 'epoch': 0.16}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.4006594717502594, 'eval_accuracy': 0.844, 'eval_f1_score': 0.8298070344212675, 'eval_runtime': 9.078, 'eval_samples_per_second': 110.156, 'eval_steps_per_second': 13.77, 'epoch': 0.16}
{'loss': 0.2544, 'grad_norm': 10.260269165039062, 'learning_rate': 2.950308815272319e-05, 'epoch': 0.2}
{'loss': 0.3306, 'grad_norm': 12.339709281921387, 'learning_rate': 2.908197641774284e-05, 'epoch': 0.24}
{'loss': 0.3338, 'grad_norm': 3.5260422229766846, 'learning_rate': 2.8660864682762495e-05, 'epoch': 0.28}
{'loss': 0.3444, 'grad_norm': 6.659819602966309, 'learning_rate': 2.8239752947782145e-05, 'epoch': 0.32}


  0%|          | 0/125 [00:00<?, ?it/s]

...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.364818274974823, 'eval_accuracy': 0.913, 'eval_f1_score': 0.9136191920985104, 'eval_runtime': 9.5374, 'eval_samples_per_second': 104.85, 'eval_steps_per_second': 13.106, 'epoch': 2.72}
{'loss': 0.1677, 'grad_norm': 0.16550901532173157, 'learning_rate': 2.5603593486805164e-06, 'epoch': 2.76}
{'loss': 0.1832, 'grad_norm': 0.25394201278686523, 'learning_rate': 2.1392476137001685e-06, 'epoch': 2.8}
{'loss': 0.1901, 'grad_norm': 0.3002878427505493, 'learning_rate': 1.7181358787198205e-06, 'epoch': 2.84}
{'loss': 0.123, 'grad_norm': 9.27582836151123, 'learning_rate': 1.2970241437394723e-06, 'epoch': 2.88}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.36782777309417725, 'eval_accuracy': 0.914, 'eval_f1_score': 0.9147457570073105, 'eval_runtime': 9.5608, 'eval_samples_per_second': 104.594, 'eval_steps_per_second': 13.074, 'epoch': 2.88}
{'loss': 0.2416, 'grad_norm': 0.05907434597611427, 'learning_rate': 8.759124087591241e-07, 'epoch': 2.92}
{'loss': 0.2024, 'grad_norm': 0.027749085798859596, 'learning_rate': 4.5480067377877594e-07, 'epoch': 2.96}
{'loss': 0.1942, 'grad_norm': 0.09840062260627747, 'learning_rate': 4.211117349803481e-08, 'epoch': 3.0}
{'train_runtime': 1380.4579, 'train_samples_per_second': 21.732, 'train_steps_per_second': 2.716, 'train_loss': 0.2445046365102132, 'epoch': 3.0}


TrainOutput(global_step=3750, training_loss=0.2445046365102132, metrics={'train_runtime': 1380.4579, 'train_samples_per_second': 21.732, 'train_steps_per_second': 2.716, 'total_flos': 3974021959680000.0, 'train_loss': 0.2445046365102132, 'epoch': 3.0})

In [17]:
# trainer.save_model("./results/distilroberta/climateBUG/seed12")
# Please remember to delete model.safetensors BEFORE adding to git. Causes issues...
# Also it is probably not worth running this block until the model is worth keeping

In [18]:
# Metrics are not included in the save model so we need to save them separately
metrics = trainer.evaluate()
with open(f"./results/distilroberta/eval_metrics_seed{seed}.json", "w") as output_file:
    json.dump(metrics, output_file)

  0%|          | 0/125 [00:00<?, ?it/s]